In [6]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.optimize as optimize
import scipy.integrate as integrate
import sympy as sp

from numba import njit

import ast

import pymc3 as pm
import arviz as az
import xarray as xr
import theano.tensor as tt

import networkx as nx

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns; sns.set_theme(style='ticks', context='paper', font_scale=0.8);

from bayern import ops

%reload_ext watermark
%watermark -a "Mathieu Baltussen" -d -t -u -v -iv

Author: Mathieu Baltussen

Last updated: 2022-04-08 16:26:22

Python implementation: CPython
Python version       : 3.9.5
IPython version      : 7.28.0

matplotlib: 3.4.2
pandas    : 1.2.4
seaborn   : 0.11.1
scipy     : 1.6.2
networkx  : 2.6.3
pymc3     : 3.11.4
arviz     : 0.11.4
numpy     : 1.20.3
xarray    : 0.19.0
bayern    : 0.1.0
sympy     : 1.8
theano    : 1.1.2



In [7]:
experiments = pd.read_csv(f"../data/kinetic_studies.csv").query(
    f'enzyme == "G6PDH"'
)
data = []
for t in experiments.itertuples():
    df = pd.read_csv(f"../data/{t.data_path}")
    df = df.assign(
        kf=t.flowrate / (60 * t.volume),
        G6PDH=t.enzyme_concentration,
        code=t.experiment_code,
    )
    data.append(df)

data = pd.concat(data).reset_index(drop=True)

exp_idx, exp_coords = data.code.factorize(sort=True)
data = data.assign(
    NAD_obs=data.NAD_in - data.NADH_obs,
    G6P_obs=data.G6P_in - data.NADH_obs,
    G6PdL_obs=data.NADH_obs,
    exp_idx = exp_idx
)
data

# data = data.groupby(['code', 'G6P_in', 'NAD_in']).mean().reset_index()


,G6P_in,NAD_in,NADH_obs,kf,G6PDH,code,NAD_obs,G6P_obs,G6PdL_obs,exp_idx
0,500,1000,324.867750,0.125,10.0,SNKS08,675.132250,175.132250,324.867750,0
1,500,1000,315.605000,0.125,10.0,SNKS08,684.395000,184.395000,315.605000,0
2,500,1000,315.956750,0.125,10.0,SNKS08,684.043250,184.043250,315.956750,0
3,1000,1000,521.965000,0.125,10.0,SNKS08,478.035000,478.035000,521.965000,0
4,1000,1000,515.164500,0.125,10.0,SNKS08,484.835500,484.835500,515.164500,0
5,1000,1000,512.233250,0.125,10.0,SNKS08,487.766750,487.766750,512.233250,0
6,1500,1000,653.871250,0.125,10.0,SNKS08,346.128750,846.128750,653.871250,0
7,1500,1000,653.285000,0.125,10.0,SNKS08,346.715000,846.715000,653.285000,0
8,1500,1000,672.748500,0.125,10.0,SNKS08,327.251500,827.251500,672.748500,0
9,3000,1000,828.339250,0.125,10.0,SNKS08,171.660750,2171.660750,828.339250,0


In [12]:
def compile_model(data):
    exp_idx, exp_coords = data.code.factorize(sort=True)
    # obs_idx, obs_coords = data.index.factorize(sort=True)
    coords = {"exp": exp_coords, 
                # 'obs': obs_coords
            }
    # print(exp_idx, exp_coords)

    with pm.Model(coords=coords) as model:
        k_cat = pm.Uniform("k_cat", 0, 500)
        K_G6P = pm.Uniform("K_G6P", 1, 4000)
        K_NAD = pm.Uniform("K_NAD", 1, 2000)
        KI_NADH = pm.Uniform("KI_NADH", 1, 10000)

        sigma = pm.Exponential("sigma", 0.5, dims='exp')

        G6PDH = pm.Data("G6PDH", data.G6PDH.values)
        NADH = pm.Data("NADH_obs", data.NADH_obs.values)
        NAD = pm.Data("NAD_obs", data.NAD_obs.values)

        G6P = pm.Data("G6P_obs", data.G6P_obs.values)
        G6PdL = pm.Data("G6PdL_obs", data.G6PdL_obs.values)

        G6P_in = pm.Data("G6P_in", data.G6P_in.values)
        NAD_in = pm.Data("NAD_in", data.NAD_in.values)
        kf = pm.Data("kf", data.kf.values)

        exp_idx = pm.Data("exp_idx", data.exp_idx.values)

        NADH_obs = pm.Normal("NADH", 
                mu = k_cat*G6PDH*G6P*NAD/(kf*K_G6P*K_NAD*(1 + G6P/K_G6P + NADH/KI_NADH)*(1+NAD/K_NAD+ NADH/KI_NADH)),
                    sigma=sigma[exp_idx],
                    observed= NADH
                    )
    return model

def select_observations(data, idx):
    return data.drop(idx), data.iloc[idx]


In [13]:
class PyMC3LinRegWrapper(az.SamplingWrapper):
    def __init__(self, data, data_vars, model, **kwargs):
        self.data = data
        self.data_vars = data_vars

        __selected_data, _ = self.sel_observations([0]) # Pre-compile model with LOO corrected shape
        self.pymc3_model = model(__selected_data)
        
        super(PyMC3LinRegWrapper, self).__init__(model=model, **kwargs)

    def sample(self, modified_observed_data):
        with self.pymc3_model:
            pm.set_data(
                modified_observed_data[self.data_vars]
            )
            trace = pm.sample(
                **self.sample_kwargs, 
                idata_kwargs={"log_likelihood": False}
            )
        return trace
    
    def get_inference_data(self, trace):
        idata = az.from_pymc3(trace, model=self.pymc3_model, **self.idata_kwargs)
        idata.pymc3_trace = trace
        return idata
        
    def log_likelihood__i(self, excluded_observed_data, idata__i):
        with self.pymc3_model:
            pm.set_data(excluded_observed_data[self.data_vars])
        
        print(excluded_observed_data[self.data_vars])
        # model_ex = compile_linreg_model(**excluded_observed_data)
        log_lik__i = az.from_pymc3(idata__i.pymc3_trace, model=self.pymc3_model, log_likelihood=True).log_likelihood["NADH"]
        return log_lik__i
        
    def sel_observations(self, idx):
        return select_observations(self.data, idx)

In [15]:
sample_kwargs = {"draws": 1000, "tune": 1000, "chains": 4, 'target_accept': 0.92, 'progressbar': True, 'return_inferencedata': False}
idata_kwargs = {
    "dims": {'NADH': ['obs']},
}
data_vars = ['G6PDH','NADH_obs','NAD_obs','G6P_obs','G6PdL_obs','G6P_in','NAD_in','kf', 'exp_idx']
with compile_model(data=data) as model:
    trace = pm.sample(**sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, KI_NADH, K_NAD, K_G6P, k_cat]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 14 seconds.


In [95]:
# inc_data, exc_data = select_observations(data, [1])

# with compile_model(data=inc_data) as model:
#     # print(model['G6PDH'].get_value())
#     # pm.set_data(exc_data[data_vars])
#     # print(model['G6PDH'].get_value())
#     trace = pm.sample(
#                     **sample_kwargs, 
#                     idata_kwargs={"log_likelihood": False}
#                 )
            
# with model:
#     pm.set_data(exc_data[data_vars])

# log_lik__i = az.from_pymc3(trace, model=model, log_likelihood=True).log_likelihood["NADH"]


In [16]:
idata = az.from_pymc3(trace, model=model, **idata_kwargs)

loo_orig = az.loo(idata, pointwise=True)
print(loo_orig)

Computed from 4000 by 48 log-likelihood matrix

         Estimate       SE
elpd_loo  -271.01    16.94
p_loo       12.26        -

There has been a warning during the calculation. Please check the results.
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)       45   93.8%
 (0.5, 0.7]   (ok)          2    4.2%
   (0.7, 1]   (bad)         1    2.1%
   (1, Inf)   (very bad)    0    0.0%



/home/mathieu/anaconda3/envs/phd/lib/python3.9/site-packages/arviz/stats/stats.py:694: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


In [17]:
pymc3_wrapper = PyMC3LinRegWrapper(
    model=compile_model, data=data, 
    data_vars=data_vars,
    sample_kwargs=sample_kwargs, idata_kwargs=idata_kwargs
)


In [18]:
loo_relooed = az.reloo(pymc3_wrapper, loo_orig=loo_orig)

/home/mathieu/anaconda3/envs/phd/lib/python3.9/site-packages/arviz/stats/stats_refitting.py:99: UserWarning: reloo is an experimental and untested feature
  warnings.warn("reloo is an experimental and untested feature", UserWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, KI_NADH, K_NAD, K_G6P, k_cat]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 12 seconds.


    G6PDH    NADH_obs     NAD_obs     G6P_obs   G6PdL_obs  G6P_in  NAD_in  \
21   10.0  1358.91066  1641.08934  1641.08934  1358.91066    3000    3000   

       kf  exp_idx  
21  0.125        0  


In [19]:
print(loo_relooed)

Computed from 4000 by 48 log-likelihood matrix

         Estimate       SE
elpd_loo  -271.15    16.99
p_loo       12.40        -

There has been a warning during the calculation. Please check the results.
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)       46   95.8%
 (0.5, 0.7]   (ok)          2    4.2%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%



In [ ]:
# exp_idx, exp_coords = data.code.factorize(sort=True)
# obs_idx, obs_coords = data.index.factorize(sort=True)
# coords = {"exp": exp_coords, 'obs': obs_coords}

# G6P, G6PdL, NAD, NADH = sym_x = sp.symbols("G6P, G6PdL, NAD, NADH")
# k_cat, K_G6P, K_NAD, KI_NADH = sym_phi = sp.symbols("k_cat, K_G6P, K_NAD, KI_NADH")
# G6PDH, G6P_in, NAD_in, kf = sym_theta = sp.symbols("G6PDH, G6P_in, NAD_in, kf")

# sym_rate = k_cat*G6PDH*G6P*NAD/(kf*K_G6P*K_NAD*(1 + G6P/K_G6P)*(1+NAD/K_NAD)*(1+ NADH/KI_NADH))
# sym_ode = [
#     -sym_rate + kf*(G6P_in - G6P),
#     sym_rate - kf*G6PdL,
#     -sym_rate + kf*(NAD_in - NAD),
#     sym_rate - kf*NADH,
# ]

# def generate_model(sym_x, sym_phi, sym_theta, sym_ode, data_set):
#     sym_jac_x = sp.Matrix(sym_ode).jacobian(sym_x)
#     sym_jac_phi = sp.Matrix(sym_ode).jacobian(sym_phi)
#     sym_jac_theta = sp.Matrix(sym_ode).jacobian(sym_theta)
#     t = sp.symbols('t')
#     num_rate_equations = njit(sp.lambdify([sym_x, sym_phi, sym_theta], sym_ode, "numpy"))
#     num_jac_x = njit(sp.lambdify([sym_x, sym_phi, sym_theta], sym_jac_x, "numpy"))
#     num_jac_phi = njit(sp.lambdify([sym_x, sym_phi, sym_theta], sym_jac_phi, "numpy"))
#     num_jac_theta = njit(sp.lambdify([sym_x, sym_phi, sym_theta], sym_jac_theta, "numpy"))

#     def find_root(fun, jac, phi, theta):
#         return optimize.root(fun=fun, x0=[theta[0],theta[1],0.0, theta[2]], jac=jac, args=(phi, theta)).x
#     num_grad_phi = njit(lambda x,phi,theta: np.dot(-np.linalg.inv(num_jac_x(x,phi,theta)),num_jac_phi(x,phi,theta)))
#     num_grad_theta = njit(lambda x,phi,theta: np.dot(-np.linalg.inv(num_jac_x(x,phi,theta)),num_jac_theta(x,phi,theta)))

#     def compile_model(xdata, ydata):
#         SteadyStateOp = ops.SteadyStateDatasetOp(num_rate_equations, num_jac_x, num_grad_phi, num_grad_theta, find_root, theta_set=xdata[sym_theta].values)

#         with pm.Model() as model:
#             x = pm.Data("x", xdata)
#             k_cat = pm.Uniform("k_cat", 0, 500)
#             K_G6P = pm.Uniform("K_G6P", 1, 4000)
#             K_NAD = pm.Uniform("K_NAD", 1, 2000)
#             KI_NADH = pm.Uniform("KI_NADH", 1, 10000)
#             sigma = pm.Exponential("sigma", 0.5, dims='exp')

#             y = pm.Normal("y", 
#                     mu=SteadyStateOp(tt.stack([k_cat, K_G6P, K_NAD, KI_NADH]))[:, 3], 
#                     sigma=sigma, 
#                     observed=ydata
#                 )
                
#         return model